In [1]:
import numpy as np
from numpy import genfromtxt
from tqdm.notebook import tqdm
from perceptronac.context_training import context_training
from perceptronac.context_coding import context_coding
from perceptronac.perfect_AC import perfect_AC
import torch
import os

In [2]:
class Perceptron(torch.nn.Module):
    def __init__(self,N):
        super().__init__()
        self.linear = torch.nn.Linear(N, 1)
        self.sigmoid = torch.nn.Sigmoid()

    def forward(self, x):
        x = self.linear(x)
        x = self.sigmoid(x)
        return x
    
class Log2BCELoss(torch.nn.Module):
    def __init__(self,*args,**kwargs):
        super().__init__()
        self.bce_loss = torch.nn.BCELoss(*args,**kwargs)

    def forward(self, pred, target):
        return self.bce_loss(pred, target)/torch.log(torch.tensor(2,dtype=target.dtype,device=target.device))
    
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self,X,y):
        self.X = X
        self.y = y
    def __len__(self):
        return len(self.y)
    def __getitem__(self,idx):
        return self.X[idx,:],self.y[idx,:]
    
class MLP_N_64N_32N_1(torch.nn.Module):
    def __init__(self,N):
        super().__init__()
        self.layers = torch.nn.Sequential(
            torch.nn.Linear(N, 64*N),
            torch.nn.ReLU(),
            torch.nn.Linear(64*N, 32*N),
            torch.nn.ReLU(),
            torch.nn.Linear(32*N, 1),
            torch.nn.Sigmoid()
        )
    def forward(self, x):
        return self.layers(x)
    
class MLP_N_1024_512_1(torch.nn.Module):
    def __init__(self,N):
        super().__init__()
        self.layers = torch.nn.Sequential(
            torch.nn.Linear(N, 1024),
            torch.nn.ReLU(),
            torch.nn.Linear(1024, 512),
            torch.nn.ReLU(),
            torch.nn.Linear(512, 1),
            torch.nn.Sigmoid()
        )
    def forward(self, x):
        return self.layers(x)


class MLP_N_2048_1024_1(torch.nn.Module):
    def __init__(self,N):
        super().__init__()
        self.layers = torch.nn.Sequential(
            torch.nn.Linear(N, 2048),
            torch.nn.ReLU(),
            torch.nn.Linear(2048, 1024),
            torch.nn.ReLU(),
            torch.nn.Linear(1024, 1),
            torch.nn.Sigmoid()
        )
    def forward(self, x):
        return self.layers(x)

## Carregando os pesos

In [7]:
file_name = 'pesos_andrew9-v3_V_1024-512.txt'
my_data_test = genfromtxt('../SPIHT_dataset/andrew/SPIHT_bits_with_context_andrew_V80_v5.csv', delimiter=',')

In [8]:
bitstream_test = my_data_test[:, 0].astype(int)
context_test = my_data_test[:, 1:6].astype(float)
extra_context_test = my_data_test[:, 6:].astype(float)
bitstream_test = bitstream_test.reshape((len(bitstream_test), 1))

yc = bitstream_test > 0 # encode the second part
Xc = context_test >= 0 # truncated X for coding
Xc = Xc.astype(int)
Xc = np.append(Xc, extra_context_test/extra_context_test.max(axis=0), axis=1)
N = Xc[0, :].size

In [9]:
model = MLP_N_1024_512_1(N)
model.load_state_dict(torch.load(file_name))

<All keys matched successfully>

## Realizando os testes

In [10]:
bit_list = [890306,507324,266178,92913,24914,5440,1829,743,293]
for i in bit_list:
    Xc_tensor = torch.from_numpy(Xc[0:i, :])
    yc_tensor = torch.from_numpy(yc[0:i])
    predictions = model(Xc_tensor.float())
    criterion=Log2BCELoss()
    tamanho_medio = criterion(predictions,yc_tensor.float())
    print(f"""comprimento médio de código final no modelo treinado: {tamanho_medio}""")

comprimento médio de código final no modelo treinado: 0.901471734046936
comprimento médio de código final no modelo treinado: 0.9125586152076721
comprimento médio de código final no modelo treinado: 0.8967228531837463
comprimento médio de código final no modelo treinado: 0.8991806507110596
comprimento médio de código final no modelo treinado: 0.8882057070732117
comprimento médio de código final no modelo treinado: 0.9351104497909546
comprimento médio de código final no modelo treinado: 0.9462733268737793
comprimento médio de código final no modelo treinado: 0.9489244818687439
comprimento médio de código final no modelo treinado: 0.9153303503990173


## Comparar com teórico

In [10]:
L = 100000 # how many samples 
#N = 10 # order of the AR
first = True
for pc_num in range(1052, 1081):
    if first:
        my_data = genfromtxt(f'../SPIHT_dataset/longdress/SPIHT_bits_with_context_longdress_Y{pc_num}_v5.csv', delimiter=',')
        first = False
    else:
        my_data = np.append(my_data,
                            genfromtxt(f'../SPIHT_dataset/longdress/SPIHT_bits_with_context_longdress_Y{pc_num}_v5.csv', delimiter=','),
                            axis=0)
    
bitstream = my_data[:, 0].astype(int)
context = my_data[:, 1:6].astype(float)
extra_context = my_data[:, 6:].astype(float)
bitstream = bitstream.reshape((len(bitstream), 1))

my_data_test = genfromtxt('../SPIHT_dataset/longdress/SPIHT_bits_with_context_longdress_Y1082_v5.csv', delimiter=',')
bitstream_test = my_data_test[:, 0].astype(int)
context_test = my_data_test[:, 1:6].astype(float)
extra_context_test = my_data_test[:, 6:].astype(float)
bitstream_test = bitstream_test.reshape((len(bitstream_test), 1))


yt = bitstream > 0 # train on the first part 
yc = bitstream_test > 0 # encode the second part
Xt = context >= 0 # truncated X for training 
Xc = context_test >= 0 # truncated X for coding
Xt = Xt.astype(int)
Xc = Xc.astype(int)
Xt = np.append(Xt, extra_context/extra_context.max(axis=0), axis=1)
Xc = np.append(Xc, extra_context_test/extra_context_test.max(axis=0), axis=1)

N = Xc[0, :].size # order of the AR

In [11]:
bit_list = [1425144,1127017,836615,530172,278762,112175,36069,10298,2800]
for i in bit_list:
    tamanho_estimado = perfect_AC(yc[0:i, :],context_coding(Xc[0:i, :],context_training(Xt,yt)))
    print(f"""comprimento médio de código final no modelo treinado: {tamanho_estimado}""")

comprimento médio de código final no modelo treinado: 0.8998248100386786
comprimento médio de código final no modelo treinado: 0.9039480465672349
comprimento médio de código final no modelo treinado: 0.9011865653318071
comprimento médio de código final no modelo treinado: 0.908467066595722
comprimento médio de código final no modelo treinado: 0.914228300718732
comprimento médio de código final no modelo treinado: 0.9237322435313927
comprimento médio de código final no modelo treinado: 0.9272066370934923
comprimento médio de código final no modelo treinado: 0.9265491010676893
comprimento médio de código final no modelo treinado: 0.9282372557795862


In [12]:
L = 100000 # how many samples 
#N = 10 # order of the AR
first = True
for pc_num in range(1052,1081):
    if first:
        my_data = genfromtxt(f'../SPIHT_dataset/longdress/SPIHT_bits_with_context_longdress_U{pc_num}_v5.csv', delimiter=',')
        first = False
    else:
        my_data = np.append(my_data,
                            genfromtxt(f'../SPIHT_dataset/longdress/SPIHT_bits_with_context_longdress_U{pc_num}_v5.csv', delimiter=','),
                            axis=0)
    
bitstream = my_data[:, 0].astype(int)
context = my_data[:, 1:6].astype(float)
extra_context = my_data[:, 6:].astype(float)
bitstream = bitstream.reshape((len(bitstream), 1))

my_data_test = genfromtxt('../SPIHT_dataset/longdress/SPIHT_bits_with_context_longdress_U1082_v5.csv', delimiter=',')
bitstream_test = my_data_test[:, 0].astype(int)
context_test = my_data_test[:, 1:6].astype(float)
extra_context_test = my_data_test[:, 6:].astype(float)
bitstream_test = bitstream_test.reshape((len(bitstream_test), 1))


yt = bitstream > 0 # train on the first part 
yc = bitstream_test > 0 # encode the second part
Xt = context >= 0 # truncated X for training 
Xc = context_test >= 0 # truncated X for coding
Xt = Xt.astype(int)
Xc = Xc.astype(int)
Xt = np.append(Xt, extra_context/extra_context.max(axis=0), axis=1)
Xc = np.append(Xc, extra_context_test/extra_context_test.max(axis=0), axis=1)

N = Xc[0, :].size # order of the AR

In [13]:
bit_list = [1073299,711315,436593,208488,84220,29887,9847,2887,816]
for i in bit_list:
    tamanho_estimado = perfect_AC(yc[0:i, :],context_coding(Xc[0:i, :],context_training(Xt,yt)))
    print(f"""comprimento médio de código final no modelo treinado: {tamanho_estimado}""")

comprimento médio de código final no modelo treinado: 0.886570537087748
comprimento médio de código final no modelo treinado: 0.9108485470670586
comprimento médio de código final no modelo treinado: 0.9105093605853859
comprimento médio de código final no modelo treinado: 0.9315149706581537
comprimento médio de código final no modelo treinado: 0.940785283571943
comprimento médio de código final no modelo treinado: 0.9414877880649091
comprimento médio de código final no modelo treinado: 0.9407770210352806
comprimento médio de código final no modelo treinado: 0.9405984261003056
comprimento médio de código final no modelo treinado: 0.9409892106836274


In [14]:
L = 100000 # how many samples 
#N = 10 # order of the AR
first = True
for pc_num in range(1052,1081):
    if first:
        my_data = genfromtxt(f'../SPIHT_dataset/longdress/SPIHT_bits_with_context_longdress_V{pc_num}_v5.csv', delimiter=',')
        first = False
    else:
        my_data = np.append(my_data,
                            genfromtxt(f'../SPIHT_dataset/longdress/SPIHT_bits_with_context_longdress_V{pc_num}_v5.csv', delimiter=','),
                            axis=0)
    
bitstream = my_data[:, 0].astype(int)
context = my_data[:, 1:6].astype(float)
extra_context = my_data[:, 6:].astype(float)
bitstream = bitstream.reshape((len(bitstream), 1))

my_data_test = genfromtxt('../SPIHT_dataset/longdress/SPIHT_bits_with_context_longdress_V1082_v5.csv', delimiter=',')
bitstream_test = my_data_test[:, 0].astype(int)
context_test = my_data_test[:, 1:6].astype(float)
extra_context_test = my_data_test[:, 6:].astype(float)
bitstream_test = bitstream_test.reshape((len(bitstream_test), 1))


yt = bitstream > 0 # train on the first part 
yc = bitstream_test > 0 # encode the second part
Xt = context >= 0 # truncated X for training 
Xc = context_test >= 0 # truncated X for coding
Xt = Xt.astype(int)
Xc = Xc.astype(int)
Xt = np.append(Xt, extra_context/extra_context.max(axis=0), axis=1)
Xc = np.append(Xc, extra_context_test/extra_context_test.max(axis=0), axis=1)

N = Xc[0, :].size # order of the AR

In [15]:
bit_list = [1073736,710427,437740,218967,96893,38163,13647,4317,1155]
for i in bit_list:
    tamanho_estimado = perfect_AC(yc[0:i, :],context_coding(Xc[0:i, :],context_training(Xt,yt)))
    print(f"""comprimento médio de código final no modelo treinado: {tamanho_estimado}""")

comprimento médio de código final no modelo treinado: 0.8886449877294255
comprimento médio de código final no modelo treinado: 0.9130558970138097
comprimento médio de código final no modelo treinado: 0.9145539491728297
comprimento médio de código final no modelo treinado: 0.9313661969952604
comprimento médio de código final no modelo treinado: 0.9416278678359963
comprimento médio de código final no modelo treinado: 0.94708922984849
comprimento médio de código final no modelo treinado: 0.9427896555366677
comprimento médio de código final no modelo treinado: 0.9339605219490211
comprimento médio de código final no modelo treinado: 0.9315936698398919
